![heart](https://www.healtheuropa.eu/wp-content/uploads/2020/12/Cardiovascular-disease-related-to-Type-2-diabetes-can-be-vastly-reduced-696x378.jpg)

Graphs gives us better idea of data but when we need exact Number we use aggregation and groupby function to get more insight from data

# Import Libraries

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

import warnings
warnings.filterwarnings("ignore")

In [ ]:
#  For using autocomplete in kaggle notebooks
%config Completer.use_jedi = False

## About Data

`age` - Age of the patient

`sex` - Sex of the patient

`cp` - Chest pain type ~ 0 = Typical Angina, 1 = Atypical Angina, 2 = Non-anginal Pain, 3 = Asymptomatic

`trtbps` - Resting blood pressure (in mm Hg)

`chol` - Cholestoral in mg/dl fetched via BMI sensor

`fbs` - (fasting blood sugar > 120 mg/dl) ~ 1 = True, 0 = False

`restecg` - Resting electrocardiographic results ~ 0 = Normal, 1 = ST-T wave normality, 2 = Left ventricular hypertrophy

`thalachh` - Maximum heart rate achieved

`oldpeak` - Previous peak

`slp` - Slope

`caa` - Number of major vessels

`thall` - Thalium Stress Test result ~ (0,3)

`exng` - Exercise induced angina ~ 1 = Yes, 0 = No

`output` - Target variable

In [ ]:
df = pd.read_csv('../input/heart-attack-analysis-prediction-dataset/heart.csv')

In [ ]:
df 

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
sns.countplot(df.output)
print(df.output.value_counts()/len(df)*100)

* Data is almost abalance

In [ ]:
sns.histplot(df.age)

In [ ]:
sns.countplot(df.sex)
print(df.sex.value_counts()/len(df)*100)

* 0 are females
* 1 is Male


It might be happen our model is more bias towards Males because it has more data

In [ ]:
sns.histplot(df.thalachh)

* thalachh = maximum heart rate achieved

In [ ]:
sns.histplot(df.chol)

* chol : cholestoral in mg/dl fetched via BMI sensor


In [ ]:
print('Before droping we have dataset of size', df.shape)
df.drop(index=df[(df.chol > 369.75) | (df.chol < 115.75)].index, inplace=True)
print('After droping we have dataset of size', df.shape)

In [ ]:
sns.countplot(df.cp)

cp : Chest Pain type chest pain type

In [ ]:
columns = ['fbs','restecg','exng','slp','caa','thall']
for col in columns:
    sns.countplot(df[col])
    plt.show()

# Multivariant analysis

## Age

`Pandas cut()` function is used to separate the array elements into different bins . The cut function is mainly used to perform statistical analysis on scalar data. 

`cut(x, bins, right=True, labels=None, retbins=False, precision=3, include_lowest=False, duplicates=”raise”,)`

Parameters:

x: The input array to be binned. Must be 1-dimensional.

bins: defines the bin edges for the segmentation.

In [ ]:
# Create bins for age and group them
# Aggregate by sum()
df.groupby(pd.cut(df.age,[25,35,45,55,65,75,85])).sum().sort_values('output', ascending=False)

In [ ]:
sns.kdeplot(df.loc[df['output'] == 0, 'age'], label = 'output 0')
sns.kdeplot(df.loc[df['output'] == 1, 'age'], label = 'output 1')
plt.legend()

In [ ]:
# Aggregate by Count()
df.groupby(pd.cut(df.age,[25,35,45,55,65,75,85])).count().sort_values('output', ascending=False)

In [ ]:
df_age = df.groupby(pd.cut(df.age,[25,35,45,55,65,75,85])).sum().sort_values('output', ascending=False)
df_age['total_count'] = df.groupby(pd.cut(df.age,[25,35,45,55,65,75,85])).count().sort_values('output', ascending=False)['output']
df_age['percentage'] = df_age['output'] / df_age['total_count'] * 100
df_age = df_age[['output','total_count','percentage']]
df_age

In [ ]:
sns.histplot(df.age,bins=6)

By above observation we don't able to say 35 - 65 have more chances of heart attacks because,

We have very less data for saying 25-35 and 65-85 has less heart attacks

So, we can't direct jump on colnclusion, we need more data

# Sex

## Male Female who is more likely to have heart problem ?

* 0 : Female
* 1 : Male

In [ ]:
df.sex.value_counts()/len(df)*100

We have inbalance dataset

In [ ]:
df.groupby('sex').sum()

By sum we can easily see Male get more heart attacks But...

In [ ]:
df.groupby('sex').mean()

Mean show completely different picture here, they show female gets more heart attack, HOW??

because male has more data points mean value decrese


and female has less data points their mean value increase

Imbalance dataset

In [ ]:
sns.kdeplot(df.loc[df['output'] == 0, 'sex'], label = 'output 0')
sns.kdeplot(df.loc[df['output'] == 1, 'sex'], label = 'output 1')
plt.legend()

* We can say Female has more probability of attacks by seeing kdeplot

In [ ]:
# copy dataframe to another varible
df_se_ag = df.copy()

# create seperate variable for both sex
df_se_ag['female'] = np.where(df_se_ag.sex == 0,1,0)
df_se_ag['male'] = np.where(df_se_ag.sex == 1,1,0)

# seperate output variable base on sex
df_se_ag['out_female'] = np.where(df_se_ag.sex == 0,df_se_ag.output,0)
df_se_ag['out_male'] = np.where(df_se_ag.sex == 1,df_se_ag.output,0)

because values are between 1 and 0 when we add them we only get count of 1

In [ ]:
# apply groupby function by age
# take only necessary columns
ma_fe_df = df_se_ag.groupby(pd.cut(df_se_ag.age,[25,35,45,55,65,75,85])).sum()[['female','male','out_female','out_male']]
ma_fe_df

`out_female` and `out_male` feature is show total number of heart attack paitent 

In [ ]:
# find out percentage 
ma_fe_df['per_female'] = ma_fe_df['out_female'] / ma_fe_df['female'] *100
ma_fe_df['per_male'] = ma_fe_df['out_male'] / ma_fe_df['male']*100

ma_fe_df

Again we confirm our previous statement by seeing agewise male female

Female has more probably of heart attacks as compare to Male

# Cholestrol

In [ ]:
df.groupby('chol').sum()

** At first glance anybody can say cholestrol above 192 is more chances of heart diseases **


There is BIG BUT..

In [ ]:
sns.kdeplot(df.loc[df['output'] == 0, 'chol'], label = 'output 0')
sns.kdeplot(df.loc[df['output'] == 1, 'chol'], label = 'output 1')
plt.legend()

In [ ]:
df_chol = df.copy()
df_chol['c_grp'] = pd.cut(df['chol'],14)
df_chol = df_chol.groupby('c_grp').sum()
df_chol['chol_count'] = df.groupby(pd.cut(df['chol'],14)).count()['chol']
df_chol['percent'] = df_chol['output'] / df_chol['chol_count']*100
df_chol

we can clearly see here cholestrol is not a good factor in Heart Disease

Because the guy who has 125 cholestrol has 66% probability and guy has more than 340 also same probability

### How many times we write same code lets make function for it

also important to write again and again to understand it

In [ ]:
def kdplot(col):
    '''
    Return
    ----------
    plot kdeplot for output variable `col`
    '''
    sns.kdeplot(df.loc[df['output'] == 0, col], label = 'output 0')
    sns.kdeplot(df.loc[df['output'] == 1, col], label = 'output 1')
    plt.legend()
    plt.show

In [ ]:
def show_per_cat(df,col):
    '''
    Function for category
    
    Return
    --------
    df_temp(Dataframe):
        Contain count of attacks , total points and percentage of attack
    '''
    df_temp = df.copy()
    df_temp = df_temp.groupby(col).sum()
    df_temp['grp_count'] = df.groupby(col).count()['output']
    df_temp['percent'] = df_temp['output'] / df_temp['grp_count']*100
    
    return df_temp[['output','grp_count','percent']]

In [ ]:
show_per_cat(df,'exng')

In [ ]:
kdplot('exng')

exercise induced angina (1 = yes; 0 = no)


0 is more probable to heart disease

# Chest Pain

In [ ]:
show_per_cat(df,'cp')

In [ ]:
kdplot('cp')

Definitely chest pain is bad sign for heart

In [ ]:
def show_per_cont(df,col,quant=5):
    '''
    Function for continuous features
    
    quant (int):
        It divide number of unique values in that feature and create those amount of bins
        we can change that if feature has many unique features 
    
    Return
    --------
    df_temp(Dataframe):
        Contain count of attacks , total points and percentage of attack
    '''
    df_chol = df.copy()
    df_chol = df_chol.groupby(pd.cut(df[col],int(df[col].nunique()/quant))).sum()
    df_chol['grp_count'] = df.groupby(pd.cut(df[col],int(df[col].nunique()/quant))).count()['output']
    df_chol['percent'] = df_chol['output'] / df_chol['grp_count']*100
    
    return df_chol[['output','grp_count','percent']]

In [ ]:
show_per_cont(df,'trtbps')

In [ ]:
kdplot('trtbps')

There is no usefulness insight from trtbps means resting blood pressure

In [ ]:
show_per_cat(df,'fbs')

In [ ]:
kdplot('fbs')

In [ ]:
show_per_cat(df,'restecg')

In [ ]:
kdplot('restecg')

Not any impactful insight

In [ ]:
show_per_cat(df,'thall')

In [ ]:
kdplot('thall')

More thall less probability to heart diseases

# Modeling

In [ ]:
# Convert features data type to category
cat_col = ['sex','cp','fbs','restecg', 'exng', 'slp', 'caa', 'thall']
df[cat_col] = df[cat_col].astype('category')

In [ ]:
# Performing One-Hot-Encoding
df_process = pd.get_dummies(df, columns=cat_col, drop_first=True)

In [ ]:
df_process

Now we have 23 features

In [ ]:
# Seperate output feature
X = df_process.drop(columns='output')
y = df_process.output

In [ ]:
# Standardise data to improve accuracy of Linear models
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X)
df_scale = scaler.transform(X)

In [ ]:
# Dividing data into train and test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [ ]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

For training we have 238 data points and for testing we have 60

Data is imbalance we cannot trust ACCURACY we use F1 score

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier


from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
target_names = ['class 0', 'class 1']

In [ ]:
lr = LogisticRegression(solver='liblinear')
lr.fit(X_train,y_train)
y_pred = lr.predict(X_test)
print(classification_report(y_test, y_pred, target_names=target_names))

In [ ]:
sv = SVC()
sv.fit(X_train,y_train)
y_pred = sv.predict(X_test)
print(classification_report(y_test, y_pred, target_names=target_names))

In [ ]:
dt = DecisionTreeClassifier()
dt.fit(X_train,y_train)
y_pred = dt.predict(X_test)
print(classification_report(y_test, y_pred, target_names=target_names))

In [ ]:
rf = RandomForestClassifier()
rf.fit(X_train,y_train)
y_pred = rf.predict(X_test)
print(classification_report(y_test, y_pred, target_names=target_names))

In [ ]:
gb = GradientBoostingClassifier(n_estimators=20)
gb.fit(X_train,y_train)
y_pred = gb.predict(X_test)
print(classification_report(y_test, y_pred, target_names=target_names))

# Hyperparameter Tunning

In [ ]:
lr = LogisticRegression()
solvers = ['newton-cg', 'lbfgs', 'liblinear']
penalty = ['l1','l2']
c_values = [100, 10, 1.0, 0.1, 0.01]
grid = dict(solver=solvers,penalty=penalty,C=c_values)
cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=1)

grid_search = GridSearchCV(estimator=lr, param_grid=grid, n_jobs=-1, cv=cv, scoring='accuracy',error_score=0)
grid_result = grid_search.fit(X, y)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

In [ ]:
lr = LogisticRegression(C=10, penalty='l2',solver='newton-cg')
lr.fit(X_train,y_train)
y_pred = lr.predict(X_test)
print(classification_report(y_test, y_pred, target_names=target_names))

In [ ]:
model = SVC()
kernel = ['poly', 'rbf', 'sigmoid']
C = [50, 10, 5, 1.0, 0.1, 0.01]
gamma = ['scale']
# define grid search
grid = dict(kernel=kernel,C=C,gamma=gamma)
cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=1)
grid_search = GridSearchCV(estimator=model, param_grid=grid, n_jobs=-1, cv=cv, scoring='accuracy',error_score=0)
grid_result = grid_search.fit(X, y)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

In [ ]:
model = RandomForestClassifier()
n_estimators = [10, 100, 1000]
max_features = ['sqrt', 'log2']
# define grid search
grid = dict(n_estimators=n_estimators,max_features=max_features)
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
grid_search = GridSearchCV(estimator=model, param_grid=grid, n_jobs=-1, cv=cv, scoring='accuracy',error_score=0)
grid_result = grid_search.fit(X, y)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

In [ ]:
# model = GradientBoostingClassifier()
# n_estimators = [10, 30, 70, 100, 1000]
# learning_rate = [0.001, 0.01, 0.1]
# subsample = [0.5, 0.7, 1.0]
# max_depth = [2, 4, 6, 8, 10]
# # define grid search
# grid = dict(learning_rate=learning_rate, n_estimators=n_estimators, subsample=subsample, max_depth=max_depth)
# cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=1)
# grid_search = GridSearchCV(estimator=model, param_grid=grid, n_jobs=-1, cv=cv, scoring='accuracy',error_score=0)
# grid_result = grid_search.fit(X, y)
# # summarize results
# print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

# >>>> Best: 0.827759 using {'learning_rate': 0.01, 'max_depth': 2, 'n_estimators': 1000, 'subsample': 0.5}

In [ ]:
gb = GradientBoostingClassifier(learning_rate=0.01, max_depth=2, n_estimators=1000, subsample=0.5)
gb.fit(X_train,y_train)
y_pred = gb.predict(X_test)
print(classification_report(y_test, y_pred, target_names=target_names))

In [ ]:
import xgboost as xgb
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.metrics import f1_score, make_scorer
import xgboost

In [ ]:
#  It takes half hour to run
# model = xgboost.XGBClassifier()
# params = {"learning_rate"    : [0.05, 0.10, 0.15],
#          "max_depth"        : [ 3, 4, 5, 6, 8, 10, 12, 15],
#           "min_child_weight" : [ 1, 3, 5, 7 ],
#           "gamma"            : [ 0.0, 0.1, 0.2 , 0.3, 0.4 ],
#           "colsample_bytree" : [ 0.3, 0.4, 0.5 , 0.7 ] }

# cv = RepeatedStratifiedKFold(n_splits=3, n_repeats=3, random_state=1)
# grid_search = RandomizedSearchCV(estimator=model, param_distributions=params, n_jobs=-1, cv=cv, scoring='accuracy',error_score=0,n_iter=20)
# grid_result = grid_search.fit(X, y)
# # summarize results
# print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

# # >>> Best: 0.813143 using {'min_child_weight': 1, 'max_depth': 3, 'learning_rate': 0.1, 'gamma': 0.3, 'colsample_bytree': 0.3}

In [ ]:
xgbt = xgboost.XGBClassifier(min_child_weight=1, max_depth=3, learning_rate=0.1, gamma=0.3, colsample_bytree=0.3)
xgbt.fit(X_train,y_train)
y_pred = xgbt.predict(X_test)
print(classification_report(y_test, y_pred, target_names=target_names))

In [ ]:
param = {'min_child_weight': 1, 'max_depth': 3, 'learning_rate': 0.1, 'gamma': 0.3, 'colsample_bytree': 0.3}
num_round = 20

In [ ]:
txgb = xgboost.XGBClassifier(min_child_weight=1, max_depth=3, learning_rate=0.1, gamma=0.3, colsample_bytree=0.3)
dtrain = xgb.DMatrix(df.drop('output',axis=1), label=df.output, enable_categorical=True)
bst = xgb.train(param, dtrain, num_round)
# txgb.fit(dtrain)
# y_pred = txgb.predict(X_test)
# print(classification_report(y_test, y_pred, target_names=target_names))

In [ ]:
# For features importance
gb = GradientBoostingClassifier(learning_rate=0.01, max_depth=2, n_estimators=1000, subsample=0.5)
gb.fit(df.drop('output',axis=1),df.output)

In [ ]:
def plot_feature_importance(importance,names,model_type):
    #Create arrays from feature importance and feature names
    feature_importance = np.array(importance)
    feature_names = np.array(names)

    #Create a DataFrame using a Dictionary
    data={'feature_names':feature_names,'feature_importance':feature_importance}
    fi_df = pd.DataFrame(data)

    #Sort the DataFrame in order decreasing feature importance
    fi_df.sort_values(by=['feature_importance'], ascending=False,inplace=True)

    #Define size of bar plot
    plt.figure(figsize=(10,8))
    #Plot Searborn bar chart
    sns.barplot(x=fi_df['feature_importance'], y=fi_df['feature_names'])
    #Add chart labels
    plt.title(model_type + 'FEATURE IMPORTANCE')
    plt.xlabel('FEATURE IMPORTANCE')
    plt.ylabel('FEATURE NAMES')


In [ ]:
plot_feature_importance(gb.feature_importances_,df.drop('output',axis=1).columns,'GRADIENT BOOST')

We see here Chest pain has maximium contribution to predictiong more chances of heart attack.

In [ ]:
# we plot feature importance
pd.Series(gb.feature_importances_,index=df.drop('output',axis=1).columns).sort_values(ascending=False)

# I hope this is HELPFUL❤️ !!!

## What next
* We can create webapp that give you advice for healthy heart from this dataset using `Flask` or `Stramlit`  